In [12]:
import re
import pymupdf
import spacy

# All task  related to Job desc

In [13]:
#extract jobdescription
# Job_desc
with open ("job_desc.txt",'r') as f:
    Job_desc = f.read()

Cleaning Jobdesc

In [14]:
# cleaning --> to lower case convert regex matching
Job_desc = Job_desc.lower()
Job_desc = re.sub('[^a-zA-Z]',' ',Job_desc)

In [44]:
#nlp -->lemmitization and stop word removing
nlp = spacy.load("en_core_web_sm")
doc_jobDesc = nlp(Job_desc)
filtered_token = [token.lemma_ for token in doc_jobDesc if not token.is_stop]
corpus_jdsc=" ".join(filtered_token)

# All task related to Resume pdf

In [16]:
doc = pymupdf.open("Avishek's resume.pdf")

In [17]:
#extract text from the pdf
out = open("output.txt","wb")
for page in doc:
    text = page.get_text().encode("utf8")
    out.write(text)
    out.write(bytes((12,)))
out.close()

In [42]:

resume_text=""
with open("output.txt","r",encoding="utf8") as f:
    resume_text = f.read()
candidate_email = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", resume_text)

Cleanig Resume text

In [19]:
#cleaning Resume text
resume_text = re.sub('[^a-zA-Z]',' ',resume_text)
resume_text = resume_text.strip()
resume_text = resume_text.lower()
resume_text = re.sub(r"[0-9]","",resume_text)
resume_text = re.sub(r"(linkedin|github)","",resume_text)
resume_text = re.sub(r"link","",resume_text)
resume_text = re.sub(r"(:|-)","",resume_text)
candidate_email = re.findall(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", resume_text)
resume_text = re.sub(r"\S+@\S+","",resume_text)
resume_text = re.sub(r"(email|gmail|mail)","",resume_text)
resume_text = re.sub(r"(contact|mobile|phone)","",resume_text)
resume_text = re.sub(r"http?s://\S+","",resume_text)
resume_text = re.sub(r"www\.\S+","",resume_text)
resume_text = re.sub(r'[•*\-+]', '', resume_text)

In [23]:
#remove stop words
doc_resume = nlp(resume_text)
token_resume = [token.lemma_ for token in doc_resume if not token.is_stop]
corpus_resume = ' '.join(token_resume)
corpus_resume = re.sub(' +',' ',corpus_resume)

# Skill matchig by spacy rule based matching

In [25]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import numpy as np

In [30]:
skill_csv = pd.read_csv("skill_list.csv") 
skill_array = np.array(skill_csv['skill'])

In [45]:
#matching skills from skill list and jobdesc
nlp_match = spacy.load("en_core_web_sm")
doc1 = nlp_match(corpus_jdsc)

matcher = Matcher(nlp_match.vocab)

# Add each skill as a separate pattern
for skill in skill_array:
    skill = skill.lower()
    pattern = [{"LOWER": skill}]
    matcher.add(skill, [pattern])  # skill name as unique matcher ID

# Run matcher
matches = matcher(doc1)

# Extract matched skills
jobdesc_skill = set()
for match_id, start, end in matches:
    skill_name = doc[start:end].text
    jobdesc_skill.add(skill_name)

In [52]:
#matching skill in jobedesc_Skill and resume
doc2 = nlp_match(corpus_resume)
matcher2 = Matcher(nlp_match.vocab)
for skill in jobdesc_skill:
    skill = skill.lower()
    pattern = [{"LOWER":skill}]
    matcher2.add(skill,[pattern])
matches2 = matcher2(doc2)
# Extract matched skills
resume_skill = set()
for string_id,start,end in matches2:
    skill_name = doc2[start:end].text
    resume_skill.add(skill_name)

# Similarty score TF-IDF and sentimeant analysis (VADER)

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
tf_idf = TfidfVectorizer()
vectors = tf_idf.fit_transform([corpus_resume, corpus_jdsc])
similarity_score = cosine_similarity(vectors[0],vectors[1])
print(similarity_score)

[[0.37576434]]


In [60]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [63]:
analyzer = SentimentIntensityAnalyzer()
vs = analyzer.polarity_scores(corpus_resume) 
score = (vs['compound']+1)/2
score

0.9907

In [86]:
total_score = 0.8*similarity_score+score*0.3
total_score [0][0]

0.5978214712880618